In [1]:
get_ipython().run_cell_magic('capture', '', "\n#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED\n#------------------------------------------\n# This is normally not required. The hub environment comes preinstaled with \n# many packages that you can already use without setup. In case there is some\n# other library you would like to use that isn't on the list you run this command\n# once to install them.  If it is already installed this command has no effect.\n!pip install db-dtypes\n!python3 -m pip install pandas\n")

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [2]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import statsmodels.api as sm
import math

In [3]:
get_ipython().run_cell_magic('capture', '', "\n#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED\n#------------------------------------------\n# This is normally not required. The hub environment comes preinstaled with \n# many packages that you can already use without setup. In case there is some\n# other library you would like to use that isn't on the list you run this command\n# once to install them.  If it is already installed this command has no effect.\n!pip install db-dtypes\n!python3 -m pip install pandas\n")

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [4]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import statsmodels.api as sm
import math

- DEFINE YOUR CLASSES AND FUNCTIONS 
-----------------------------------
This is not required, but is helpful in keeping your notebook organized. 
You can use the following cell or several cells to define your functions
and classes to keep them separate from your analysis or results code.
In general it useful to define your methods in a separate cell from where
it is run.

In [5]:
def dataExplore(data):
    '''
    Explore dataframe
    '''
    print("# of observations: ", data.shape[0])
    for col in data.columns:
        if col in ["uu_id", "timeperiod", "week_number", "countyfips", "tract", "tract_name", "date"]:
            print("# of %s: %s" % (col, len(pd.unique(data[col]))))
        else:
            print("Unique value of %s: %s" % (col, pd.unique(data[col])))

In [6]:
def dataBalanceCheck(data):
    '''
    Check the balance of data frame
    '''
    unbalance_count = 0
    print("# of observations in complete time series: ", len(pd.unique(data["week_number"])))
    for id in pd.unique(data["uu_id"]):
        if len(data[data["uu_id"] == id]) <  len(pd.unique(data["week_number"])):
            print(id, len(data[data["uu_id"] == id]))
            unbalance_count += 1
    print("% of tracts with incomplete time series: ", unbalance_count / len(pd.unique(data["uu_id"]))*100)

In [7]:
def dataFillNa(data, value):
    """
   fill NA with given value in the dataframe
    """
    for col in data.columns:
        if col in ["uu_id", "timeperiod", "week_number", "countyfips", "tract", "tract_name", "date"]:
            pass
        elif col in ["top_category_employer1", "top_category_employer2", "top_category_employer3"]:
            data[col] = data[col].replace({'N/A':str(value)})
        else:
            data[col] = data[col].fillna(value)
    return(data)

In [8]:
def dataIdentifyDWM(data):
    '''
    Input: # of week. Output: data for the first day, its month and week order in the month
    '''
    data["date"] = pd.to_datetime(2022 * 1000 + (1+(data["week_number"]-1)*7), format='%Y%j')
    data["month"] = pd.DatetimeIndex(data["date"]).month
    data["weekofmonth"]= pd.to_numeric(data["date"].dt.day/7)
    data['weekofmonth'] = data['weekofmonth'].apply(lambda x: math.ceil(x))
    return(data)

In [9]:
def MSPE(s1, s2):
    print("MSPE: ", sum((s1 - s2)**2)/len(s1))

In [10]:
def MAPE(s1, s2):
    print("MAPE: ", sum(abs(s1 - s2))/len(s1))

In [11]:
# Obtain data using BigQuery
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [12]:
query = """
SELECT
a.*,
b.average_wage
FROM 
(SELECT 
*
FROM `ironhacks-data.ironhacks_competition.unemployment_data`) a
JOIN `ironhacks-data.ironhacks_competition.wage_data` b 
ON a.uu_id=b.uu_id
"""

In [13]:
query_job = bigquery_client.query(query)
data = query_job.to_dataframe()

In [14]:
query_pred = """
SELECT * FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [15]:
query_job_pred = bigquery_client.query(query_pred)
data_pred_query= query_job_pred.to_dataframe()

In [16]:
# Further check tracts with average_wage as Nan
# I find three tracts with all average_wage as Nan. If I drop these tracts due to Nan value, they cannot be predicted
for id in pd.unique(data[data['average_wage'].isna()]["uu_id"]):
    print(id)
    print("All value are nan?", data[data['uu_id'] == id]["average_wage"].isnull().all())
    print("Included in prediction list?", len(data_pred_query[data_pred_query['uu_id'] == id]) > 0)

In [17]:
# Backup the data before pre-treatment
data_backup = data.copy()
data_pred_query_backup = data_pred_query.copy()

In [18]:
# Pretreatment: convert week_number to month and week of month, to capture seasonality
data = dataIdentifyDWM(data)

In [19]:
# To balance the dataset as panel data
data_balance = data.set_index('week_number')
data_balance = data_balance.sort_index(ascending=False)
data_balance = data_balance.set_index('uu_id',append=True)
data_balance = data_balance[~data_balance.index.duplicated(keep='first')]

In [20]:
data_balance = data_balance.reset_index(level=['week_number'])
data_balance = (data_balance.set_index('week_number',append=True).reindex(pd.MultiIndex.from_product([data_balance.index.unique(),
                                                      range(data_balance.week_number.min(),data_balance.week_number.max()+1)],
                                                     names=['uu_id','week_number'])).reset_index(level=1))

In [21]:
data_balance = data_balance.set_index('week_number',append=True)
data_balance['total_claims'] = data_balance['total_claims'].fillna(0)
data_balance['average_wage'] = data_balance['average_wage'].interpolate(method = "linear")

In [22]:
data_balance = data_balance.reset_index(level=['uu_id', "week_number"])
data_balance = dataIdentifyDWM(data_balance)

In [23]:
dataBalanceCheck(data_balance)

# of observations in complete time series:  37


% of tracts with incomplete time series:  0.0


In [24]:
# Data clean up: convert NA to 0 for gender, race, education and top employer and recalculate unknown category
# Based on the check of Nan in average_wage above, I also convert Nan to zero as well, but try models with and without "average_wage" variable
data = dataFillNa(data, 0)

In [25]:
# Split data to training and validaton sets
train_week = max(pd.unique(data["week_number"]))

In [26]:
data_train = data[data["week_number"] < train_week]
data_valid = data[data["week_number"] >= train_week]

In [27]:
data_train_x = data_train.drop("total_claims",1)
data_train_y = data_train["total_claims"]

/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/3749289176.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_train_x = data_train.drop("total_claims",1)


In [28]:
data_valid_x = data_valid.drop("total_claims",1)
data_valid_y = data_valid["total_claims"]

/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/3745287416.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_valid_x = data_valid.drop("total_claims",1)


Also prepare it for balanced dataset

In [29]:
data_balance_train = data_balance[data_balance["week_number"] < train_week]
data_balance_valid = data_balance[data_balance["week_number"] >= train_week]

In [30]:
data_balance_train_x = data_balance_train.drop("total_claims",1)
data_balance_train_y = data_balance_train["total_claims"]

/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/246331275.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_balance_train_x = data_balance_train.drop("total_claims",1)


In [31]:
data_balance_valid_x = data_balance_valid.drop("total_claims",1)
data_balance_valid_y = data_balance_valid["total_claims"]

/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/3492149196.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_balance_valid_x = data_balance_valid.drop("total_claims",1)


In [32]:
# Model 1 : Poisson regression with unbalanced data
data_train_x_m1 = data_train_x[["week_number","month", "weekofmonth",  "average_wage"]]
data_train_x_m1["month"] = data_train_x_m1["month"].astype(str)
data_train_x_m1["weekofmonth"] = data_train_x_m1["weekofmonth"].astype(str)
data_train_x_m1["week_number2"] = data_train_x_m1["week_number"]**2
data_train_x_m1 = pd.get_dummies(data_train_x_m1)

/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/4175230888.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_x_m1["month"] = data_train_x_m1["month"].astype(str)
/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/4175230888.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_x_m1["weekofmonth"] = data_train_x_m1["weekofmonth"].astype(str)
/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/4175230888.py:5: SettingWithCopyWarning: 
A value i

In [33]:
data_valid_x_m1 = data_valid_x[["week_number","month", "weekofmonth",  "average_wage"]]
data_valid_x_m1["month"] = data_valid_x_m1["month"].astype(str)
data_valid_x_m1["weekofmonth"] = data_valid_x_m1["weekofmonth"].astype(str)
data_valid_x_m1["week_number2"] = data_valid_x_m1["week_number"]**2
data_valid_x_m1 = pd.get_dummies(data_valid_x_m1)

/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/997101271.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_valid_x_m1["month"] = data_valid_x_m1["month"].astype(str)
/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/997101271.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_valid_x_m1["weekofmonth"] = data_valid_x_m1["weekofmonth"].astype(str)
/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/997101271.py:4: SettingWithCopyWarning: 
A value is t

In [34]:
for i in range(8):
    data_valid_x_m1["month_"+str(1+i)] = 0

In [35]:
for i in range(5):
    if i == 1:
        pass
    data_valid_x_m1["weekofmonth_"+str(1+i)] = 0

In [36]:
poission_model = sm.GLM(data_train_y.astype(int), data_train_x_m1.astype(float), family=sm.families.Poisson())
result = poission_model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           total_claims   No. Observations:                15987
Model:                            GLM   Df Residuals:                    15972
Model Family:                 Poisson   Df Model:                           14
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -91741.
Date:                Tue, 09 May 2023   Deviance:                   1.0617e+05
Time:                        22:23:56   Pearson chi2:                 1.33e+05
No. Iterations:                     5   Pseudo R-squ. (CS):             0.2356
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
week_number       0.0995      0.002     47.746      0.000       0.095       0.104
average_wage  -1.341e-05   4.17e-07    -32.198      0.000   -1.42e-05   -1.26e-05
week_number2     -0.0004   6.42e-05     -6.565      0.000      -0.001      -0.000
month_1           2.1226      0.005    406.869      0.000       2.112       2.133
month_2           1.6196      0.009    187.307      0.000       1.603       1.637
month_3           1.2621      0.012    108.911      0.000       1.239       1.285
month_4           0.9807      0.012     79.097      0.000       0.956       1.005
month_5           0.3220      0.012     26.872      0.000       0.299       0.346
month_6          -0.0113      0.009     -1.193      0.233      -0.030       0.007
month_7          -0.2598      0.005    -51.383      0.000      -0.270      -0.250
month_8          -0.6277      0.008    -75.898      0.000      -0.644      -0.611
month_9          -0.7077      0.015    -46.552      0.000      -0.737      -0.678
weekofmonth_1     1.0676      0.007    156.084      0.000       1.054       1.081
weekofmonth_2     1.0678      0.006    168.216      0.000       1.055       1.080
weekofmonth_3     0.9275      0.006    150.945      0.000       0.915       0.940
weekofmonth_4     0.8476      0.006    149.460      0.000       0.836       0.859
weekofmonth_5     0.7901      0.008    101.001      0.000       0.775       0.805
=================================================================================
"""

In [37]:
data_estimate_m1 = result.predict(data_valid_x_m1.astype(float))
MSPE(data_estimate_m1, data_valid_y)
MAPE(data_estimate_m1, data_valid_y)

MSPE:  18014.47202658045
MAPE:  132.17734114950738


In [38]:
# Model 2: Poisson with balanced data
data_balance_train_x_m1 = data_balance_train_x[["week_number","month", "weekofmonth",  "average_wage"]]
data_balance_train_x_m1["month"] = data_balance_train_x_m1["month"].astype(str)
data_balance_train_x_m1["weekofmonth"] = data_balance_train_x_m1["weekofmonth"].astype(str)
data_balance_train_x_m1["week_number2"] = data_balance_train_x_m1["week_number"]**2
data_balance_train_x_m1 = pd.get_dummies(data_balance_train_x_m1)

/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/544760508.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_balance_train_x_m1["month"] = data_balance_train_x_m1["month"].astype(str)
/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/544760508.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_balance_train_x_m1["weekofmonth"] = data_balance_train_x_m1["weekofmonth"].astype(str)
/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/544760508.py:5: Setti

In [39]:
data_balance_valid_x_m1 = data_balance_valid_x[["week_number","month", "weekofmonth",  "average_wage"]]
data_balance_valid_x_m1["month"] = data_balance_valid_x_m1["month"].astype(str)
data_balance_valid_x_m1["weekofmonth"] = data_balance_valid_x_m1["weekofmonth"].astype(str)
data_balance_valid_x_m1["week_number2"] = data_balance_valid_x_m1["week_number"]**2
data_balance_valid_x_m1 = pd.get_dummies(data_balance_valid_x_m1)

/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/236819202.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_balance_valid_x_m1["month"] = data_balance_valid_x_m1["month"].astype(str)
/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/236819202.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_balance_valid_x_m1["weekofmonth"] = data_balance_valid_x_m1["weekofmonth"].astype(str)
/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/236819202.py:4: Setti

In [40]:
for i in range(8):
    data_balance_valid_x_m1["month_"+str(1+i)] = 0

In [41]:
for i in range(5):
    if i == 1:
        pass
    data_balance_valid_x_m1["weekofmonth_"+str(1+i)] = 0

In [42]:
poission_model_m2 = sm.GLM(data_balance_train_y.astype(int), data_balance_train_x_m1.astype(float), family=sm.families.Poisson())
result_m2 = poission_model_m2.fit()
result_m2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           total_claims   No. Observations:                18900
Model:                            GLM   Df Residuals:                    18885
Model Family:                 Poisson   Df Model:                           14
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.7382e+05
Date:                Tue, 09 May 2023   Deviance:                   2.8202e+05
Time:                        22:23:56   Pearson chi2:                 2.76e+05
No. Iterations:                     5   Pseudo R-squ. (CS):             0.6626
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
week_number      -0.0230      0.002     -9.760      0.000      -0.028      -0.018
average_wage  -2.722e-05   4.49e-07    -60.632      0.000   -2.81e-05   -2.63e-05
week_number2      0.0032   7.26e-05     44.588      0.000       0.003       0.003
month_1           2.0364      0.005    387.843      0.000       2.026       2.047
month_2           2.0818      0.009    225.369      0.000       2.064       2.100
month_3           1.8232      0.013    143.545      0.000       1.798       1.848
month_4           1.6441      0.014    118.539      0.000       1.617       1.671
month_5           0.8062      0.013     61.293      0.000       0.780       0.832
month_6           0.0769      0.010      7.472      0.000       0.057       0.097
month_7          -0.2897      0.005    -53.410      0.000      -0.300      -0.279
month_8          -0.9624      0.010    -92.404      0.000      -0.983      -0.942
month_9          -1.2666      0.017    -74.508      0.000      -1.300      -1.233
weekofmonth_1     1.2472      0.007    175.029      0.000       1.233       1.261
weekofmonth_2     1.3475      0.007    189.744      0.000       1.334       1.361
weekofmonth_3     1.2469      0.007    177.009      0.000       1.233       1.261
weekofmonth_4     0.9476      0.007    138.124      0.000       0.934       0.961
weekofmonth_5     1.1608      0.008    145.799      0.000       1.145       1.176
=================================================================================
"""

In [43]:
data_balance_estimate_m2 = result_m2.predict(data_balance_valid_x_m1.astype(float))
MSPE(data_balance_estimate_m2, data_balance_valid_y)
MAPE(data_balance_estimate_m2, data_balance_valid_y)

MSPE:  33104.78877060478
MAPE:  179.04910982927015


In [44]:
# Although using a balanced model has better fit on the training set, the MSPE and MAPE are still larger then the first model.
# So for this submission, I sitll use the m1 for prediction.
data_lastWeek = data[data["week_number"] == train_week][["uu_id", "average_wage"]]
data_lastWeek = data_lastWeek.drop_duplicates()

In [45]:
data_pred = data_pred_query.copy()
data_pred = data_pred.set_index('uu_id').join(data_lastWeek.set_index('uu_id'))
data_pred.head()
data_pred = dataIdentifyDWM(data_pred)
data_pred = dataFillNa(data_pred, 0)
data_pred["month"] = data_pred["month"].astype(str)
data_pred = data_pred.drop("date",1)
data_pred = pd.get_dummies(data_pred)
for i in range(8):
    data_pred["month_"+str(1+i)] = 0
for i in range(5):
    if i == 1:
        pass
    data_pred["weekofmonth_"+str(1+i)] = 0

/var/folders/l2/cnhff0_n1470l4xf0p7v7l_40000gn/T/ipykernel_18169/2532250046.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_pred = data_pred.drop("date",1)


In [46]:
output = result.predict(data_pred.astype(float))
output_df = pd.DataFrame(output, columns=["total_claims"])["total_claims"]
output_df["uu_id"] = output_df.index

In [47]:
data_pred_query = data_pred_query.merge(output_df, on = "uu_id")
data_pred_query = data_pred_query[["uu_id", "total_claims", "week_number"]]

In [48]:
## This can also be a good place for you to cleanup any input/output and export your results to a file.
data_pred_query.to_csv("submission_prediction_output.csv", index=False)